# 1. Import Libraries


In [ ]:
import torch
from torch import Tensor, nn
import torch.nn.functional as F
import torchvision.models as models
from typing import Any, Optional, Tuple, Type
from torch.autograd import Variable
from torch.optim import AdamW
from torch.optim.lr_scheduler import MultiStepLR


from torchvision import transforms
import os
import gc
import time
import copy
import xml.etree.ElementTree as ET
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import InterpolationMode


import numpy as np
import pandas as pd
from scipy.integrate import simpson as simps
import math
from torchvision.models import swin_b
from tqdm import tqdm

In [2]:
# [LOG]
print(">> [1] After importing libraries!!!!")

>> [1] After importing libraries!!!!


# 2. Transformer


In [3]:
class MLPBlock(nn.Module):
    """ MLP Block for Transformer """

    def __init__(self,
                 embedding_dim: int,
                 mlp_dim: int,
                 activation: Type[nn.Module] = nn.GELU) -> None:
        super().__init__()

        self.linear_1 = nn.Linear(embedding_dim, mlp_dim)
        self.linear_2 = nn.Linear(mlp_dim, embedding_dim)
        self.activation = activation()

    def forward(self, x: Tensor) -> Tensor:
        x = self.linear_1(x)
        x = self.activation(x)
        x = self.linear_2(x)
        return x

In [4]:
# From https://github.com/facebookresearch/detectron2/blob/main/detectron2/layers/batch_norm.py # noqa
# Itself from https://github.com/facebookresearch/ConvNeXt/blob/d1fa8f6fef0a165b27399986cc2bdacc92777e40/models/convnext.py#L119  # noqa
class LayerNorm2d(nn.Module):

    def __init__(self, num_channels: int, eps: float = 1e-6) -> None:
        super().__init__()
        self.weight = nn.Parameter(torch.ones(num_channels))
        self.bias = nn.Parameter(torch.zeros(num_channels))
        self.eps = eps

    def forward(self, x: Tensor) -> Tensor:
        u = x.mean(1, keepdim=True)
        s = (x - u).pow(2).mean(1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.eps)
        x = self.weight[:, None, None] * x + self.bias[:, None, None]
        return x

In [5]:
class Attention(nn.Module):
    """
    An attention layer that allows for downscaling the size of the embedding
    after projection to queries, keys, and values.
    """

    def __init__(
        self,
        embedding_dim: int,
        num_heads: int,
        downsample_rate: int = 1,
    ) -> None:
        super().__init__()
        self.embedding_dim = embedding_dim
        self.internal_dim = embedding_dim // downsample_rate
        self.num_heads = num_heads
        assert self.internal_dim % num_heads == 0, "num_heads must divide embedding_dim."

        self.q_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.k_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.v_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.out_proj = nn.Linear(self.internal_dim, embedding_dim)

    def _separate_heads(self, x: Tensor, num_heads: int) -> Tensor:
        b, n, c = x.shape
        x = x.reshape(b, n, num_heads, c // num_heads)
        return x.transpose(1, 2)  # B x N_heads x N_tokens x C_per_head

    def _recombine_heads(self, x: Tensor) -> Tensor:
        b, n_heads, n_tokens, c_per_head = x.shape
        x = x.transpose(1, 2)
        return x.reshape(b, n_tokens, n_heads * c_per_head)  # B x N_tokens x C

    def forward(self, q: Tensor, k: Tensor, v: Tensor) -> Tensor:
        # Input projections
        q = self.q_proj(q)
        k = self.k_proj(k)
        v = self.v_proj(v)

        # Separate into heads
        q = self._separate_heads(q, self.num_heads)
        k = self._separate_heads(k, self.num_heads)
        v = self._separate_heads(v, self.num_heads)

        # Attention
        _, _, _, c_per_head = q.shape
        attn = q @ k.permute(0, 1, 3, 2)  # B x N_heads x N_tokens x N_tokens
        attn = attn / math.sqrt(c_per_head)
        attn = torch.softmax(attn, dim=-1)

        # Get output
        out = attn @ v
        out = self._recombine_heads(out)
        out = self.out_proj(out)

        return out

In [6]:
class TwoWayAttentionBlock(nn.Module):

    def __init__(
        self,
        embedding_dim: int,
        num_heads: int,
        mlp_dim: int = 2048,
        activation: Type[nn.Module] = nn.ReLU,
        attention_downsample_rate: int = 2,
        skip_first_layer_pe: bool = False,
    ) -> None:
        """
        A transformer block with four layers: (1) self-attention of sparse
        inputs, (2) cross attention of sparse inputs to dense inputs, (3) mlp
        block on sparse inputs, and (4) cross attention of dense inputs to sparse
        inputs.

        Arguments:
          embedding_dim (int): the channel dimension of the embeddings
          num_heads (int): the number of heads in the attention layers
          mlp_dim (int): the hidden dimension of the mlp block
          activation (nn.Module): the activation of the mlp block
          skip_first_layer_pe (bool): skip the PE on the first layer
        """
        super().__init__()
        self.self_attn = Attention(embedding_dim, num_heads)
        self.norm1 = nn.LayerNorm(embedding_dim)

        self.cross_attn_token_to_image = Attention(
            embedding_dim, num_heads,
            downsample_rate=attention_downsample_rate
        )
        self.norm2 = nn.LayerNorm(embedding_dim)

        self.mlp = MLPBlock(embedding_dim, mlp_dim, activation)
        self.norm3 = nn.LayerNorm(embedding_dim)

        self.cross_attn_image_to_token = Attention(
            embedding_dim, num_heads,
            downsample_rate=attention_downsample_rate
        )
        self.norm4 = nn.LayerNorm(embedding_dim)

        self.skip_first_layer_pe = skip_first_layer_pe

    def forward(
        self, queries: Tensor, keys: Tensor, query_pe: Tensor, key_pe: Tensor
    ) -> Tuple[Tensor, Tensor]:
        # Self attention block
        if self.skip_first_layer_pe:
            queries = self.self_attn(q=queries, k=queries, v=queries)
        else:
            q = queries + query_pe
            attn_out = self.self_attn(q=q, k=q, v=queries)
            queries = queries + attn_out
        queries = self.norm1(queries)

        # Cross attention block, tokens attending to image embedding
        q = queries + query_pe
        k = keys + key_pe
        attn_out = self.cross_attn_token_to_image(q=q, k=k, v=keys)
        queries = queries + attn_out
        queries = self.norm2(queries)

        # MLP block
        mlp_out = self.mlp(queries)
        queries = queries + mlp_out
        queries = self.norm3(queries)

        # Cross attention block, image embedding attending to tokens
        q = queries + query_pe
        k = keys + key_pe
        attn_out = self.cross_attn_image_to_token(q=k, k=q, v=queries)
        keys = keys + attn_out
        keys = self.norm4(keys)

        return queries, keys

In [7]:
class TwoWayTransformer(nn.Module):

    def __init__(
        self,
        depth: int,
        embedding_dim: int,
        num_heads: int,
        mlp_dim: int,
        activation: Type[nn.Module] = nn.ReLU,
        attention_downsample_rate: int = 2,
    ) -> None:
        """
        A transformer decoder that attends to an input image using
        queries whose positional embedding is supplied.

        Args:
          depth (int): number of layers in the transformer
          embedding_dim (int): the channel dimension for the input embeddings
          num_heads (int): the number of heads for multihead attention. Must
            divide embedding_dim
          mlp_dim (int): the channel dimension internal to the MLP block
          activation (nn.Module): the activation to use in the MLP block
        """
        super().__init__()
        self.depth = depth
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.layers = nn.ModuleList()

        for i in range(depth):
            self.layers.append(
                TwoWayAttentionBlock(
                    embedding_dim=embedding_dim,
                    num_heads=num_heads,
                    mlp_dim=mlp_dim,
                    activation=activation,
                    attention_downsample_rate=attention_downsample_rate,
                    skip_first_layer_pe=(i == 0),
                )
            )

        self.final_attn_token_to_image = Attention(
            embedding_dim, num_heads,
            downsample_rate=attention_downsample_rate
        )
        self.norm_final_attn = nn.LayerNorm(embedding_dim)

    def forward(
        self,
        image_embedding: Tensor,
        image_pe: Tensor,
        point_embedding: Tensor,
    ) -> Tuple[Tensor, Tensor]:
        """
        Args:
          image_embedding (torch.Tensor): image to attend to. Should be shape
            B x embedding_dim x h x w for any h and w.
          image_pe (torch.Tensor): the positional encoding to add to the image. Must
            have the same shape as image_embedding.
          point_embedding (torch.Tensor): the embedding to add to the query points.
            Must have shape B x N_points x embedding_dim for any N_points.

        Returns:
          torch.Tensor: the processed point_embedding
          torch.Tensor: the processed image_embedding
        """
        # BxCxHxW -> BxHWxC == B x N_image_tokens x C
        bs, c, h, w = image_embedding.shape
        image_embedding = image_embedding.flatten(2).permute(0, 2, 1)
        image_pe = image_pe.flatten(2).permute(0, 2, 1)

        # Prepare queries
        queries = point_embedding
        keys = image_embedding

        # Apply transformer blocks and final layernorm
        for layer in self.layers:
            queries, keys = layer(
                queries=queries,
                keys=keys,
                query_pe=point_embedding,
                key_pe=image_pe,
            )

        # Apply the final attention layer from the points to the image
        q = queries + point_embedding
        k = keys + image_pe
        attn_out = self.final_attn_token_to_image(q=q, k=k, v=keys)
        queries = queries + attn_out
        queries = self.norm_final_attn(queries)

        return queries, keys

In [8]:
# [LOG]
print(">> [2] After defining Transformer classes!!!!")

>> [2] After defining Transformer classes!!!!


# 3. FaceXFormer


In [9]:
class MLP(nn.Module):

    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
        num_layers: int,
        sigmoid_output: bool = False,
    ) -> None:
        super().__init__()
        self.num_layers = num_layers
        h = [hidden_dim] * (num_layers - 1)
        self.layers = nn.ModuleList(
            nn.Linear(n, k) for n, k in zip([input_dim] + h, h + [output_dim])
        )
        self.sigmoid_output = sigmoid_output

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = F.relu(layer(x)) if i < self.num_layers - 1 else layer(x)
        if self.sigmoid_output:
            x = F.sigmoid(x)
        return x

In [10]:
class FaceDecoder(nn.Module):

    def __init__(
        self,
        *,
        transformer_dim: 256,
        transformer: nn.Module,
        activation: Type[nn.Module] = nn.GELU,
    ) -> None:

        super().__init__()
        self.transformer_dim = transformer_dim
        self.transformer = transformer

        self.landmarks_token = nn.Embedding(1, transformer_dim)

        self.landmarks_prediction_head = MLP(
            transformer_dim, transformer_dim, 136, 3
        )

    def forward(
        self,
        image_embeddings: Tensor,
        image_pe: Tensor,
    ) -> Tensor:
        output_tokens = torch.cat([self.landmarks_token.weight], dim=0)
        tokens = output_tokens.unsqueeze(0).expand(
            image_embeddings.size(0), -1, -1)

        src = image_embeddings
        pos_src = image_pe.expand(image_embeddings.size(0), -1, -1, -1)
        b, c, h, w = src.shape

        hs, src = self.transformer(src, pos_src, tokens)

        landmarks_token_out = hs[:, 0, :]

        landmark_output = self.landmarks_prediction_head(landmarks_token_out)

        return landmark_output

In [11]:
class PositionEmbeddingRandom(nn.Module):
    """
    Positional encoding using random spatial frequencies.
    """

    def __init__(self, num_pos_feats: int = 64, scale: Optional[float] = None) -> None:
        super().__init__()
        if scale is None or scale <= 0.0:
            scale = 1.0
        self.register_buffer(
            "positional_encoding_gaussian_matrix",
            scale * torch.randn((2, num_pos_feats)),
        )

    def _pe_encoding(self, coords: torch.Tensor) -> torch.Tensor:
        """Positionally encode points that are normalized to [0,1]."""
        # assuming coords are in [0, 1]^2 square and have d_1 x ... x d_n x 2 shape
        coords = 2 * coords - 1
        coords = coords @ self.positional_encoding_gaussian_matrix
        coords = 2 * np.pi * coords
        # outputs d_1 x ... x d_n x C shape
        return torch.cat([torch.sin(coords), torch.cos(coords)], dim=-1)

    def forward(self, size: Tuple[int, int]) -> torch.Tensor:
        """Generate positional encoding for a grid of the specified size."""
        h, w = size
        device: Any = self.positional_encoding_gaussian_matrix.device
        grid = torch.ones((h, w), device=device, dtype=torch.float32)
        y_embed = grid.cumsum(dim=0) - 0.5
        x_embed = grid.cumsum(dim=1) - 0.5
        y_embed = y_embed / h
        x_embed = x_embed / w

        pe = self._pe_encoding(torch.stack([x_embed, y_embed], dim=-1))
        return pe.permute(2, 0, 1)  # C x H x W

    def forward_with_coords(
        self, coords_input: torch.Tensor, image_size: Tuple[int, int]
    ) -> torch.Tensor:
        """Positionally encode points that are not normalized to [0,1]."""
        coords = coords_input.clone()
        coords[:, :, 0] = coords[:, :, 0] / image_size[1]
        coords[:, :, 1] = coords[:, :, 1] / image_size[0]
        return self._pe_encoding(coords.to(torch.float))  # B x N x C

In [12]:
class FaceXFormerMLP(nn.Module):

    def __init__(self, input_dim):
        super().__init__()
        self.proj = nn.Linear(input_dim, 256)

    def forward(self, hidden_states: torch.Tensor):
        hidden_states = hidden_states.flatten(2).transpose(1, 2)
        hidden_states = self.proj(hidden_states)
        return hidden_states

In [13]:
class FaceXFormer(nn.Module):

    def __init__(self, backbone_name: str):
        super(FaceXFormer, self).__init__()

        if backbone_name != 'swin_b':
            raise ValueError(f"Unsupported backbone name: {backbone_name}")

        swin_v2 = swin_b(weights='IMAGENET1K_V1')
        self.backbone = torch.nn.Sequential(*(list(swin_v2.children())[:-1]))
        self.target_layer_names = ['0.1', '0.3', '0.5', '0.7']
        self.multi_scale_features = []

        embed_dim = 1024
        out_chans = 256

        self.pe_layer = PositionEmbeddingRandom(out_chans // 2)

        for name, module in self.backbone.named_modules():
            if name in self.target_layer_names:
                module.register_forward_hook(self.save_features_hook(name))

        self.face_decoder = FaceDecoder(
            transformer_dim=256,
            transformer=TwoWayTransformer(
                depth=2,
                embedding_dim=256,
                mlp_dim=2048,
                num_heads=8,
            ))

        num_encoder_blocks = 4
        hidden_sizes = [128, 256, 512, 1024]
        decoder_hidden_size = 256

        mlps = []
        for i in range(num_encoder_blocks):
            mlp = FaceXFormerMLP(input_dim=hidden_sizes[i])
            mlps.append(mlp)
        self.linear_c = nn.ModuleList(mlps)

        self.linear_fuse = nn.Conv2d(
            in_channels=decoder_hidden_size * num_encoder_blocks,
            out_channels=decoder_hidden_size,
            kernel_size=1,
            bias=False,
        )

    def save_features_hook(self, name):
        def hook(module, input, output):
            self.multi_scale_features.append(
                output.permute(0, 3, 1, 2).contiguous())
        return hook

    def forward(self, x, labels, tasks):
        self.multi_scale_features.clear()

        _, _, h, w = x.shape
        features = self.backbone(x).squeeze()

        batch_size = self.multi_scale_features[-1].shape[0]
        all_hidden_states = ()
        for encoder_hidden_state, mlp in zip(self.multi_scale_features, self.linear_c):

            height, width = encoder_hidden_state.shape[2], encoder_hidden_state.shape[3]
            encoder_hidden_state = mlp(encoder_hidden_state)
            encoder_hidden_state = encoder_hidden_state.permute(0, 2, 1)
            encoder_hidden_state = encoder_hidden_state.reshape(
                batch_size, -1, height, width)
            encoder_hidden_state = nn.functional.interpolate(
                encoder_hidden_state, size=self.multi_scale_features[0].size()[
                    2:],
                mode="bilinear", align_corners=False
            )
            all_hidden_states += (encoder_hidden_state,)

        fused_states = self.linear_fuse(
            torch.cat(all_hidden_states[::-1], dim=1))
        image_pe = self.pe_layer(
            (fused_states.shape[2], fused_states.shape[3])).unsqueeze(0)

        landmark_output = self.face_decoder(
            image_embeddings=fused_states,
            image_pe=image_pe
        )

        return landmark_output

In [14]:
# [LOG]
print(">> [3] After defining FaceXFormer model!!!!")

>> [3] After defining FaceXFormer model!!!!


# 4. Loss functions


## 4.1. WingLoss


In [15]:
# Wing Loss
class WingLoss(nn.Module):

    def __init__(self, omega=0.01, epsilon=2):
        super(WingLoss, self).__init__()
        self.omega = omega
        self.epsilon = epsilon

    def forward(self, pred, target):
        y = target
        y_hat = pred
        delta_2 = (y - y_hat).pow(2).sum(dim=-1, keepdim=False)
        # delta = delta_2.sqrt()
        delta = delta_2.clamp(min=1e-6).sqrt()
        C = self.omega - self.omega * math.log(1 + self.omega / self.epsilon)
        loss = torch.where(
            delta < self.omega,
            self.omega * torch.log(1 + delta / self.epsilon),
            delta - C
        )
        return loss.mean()

## 4.2. SmoothL1Loss


In [16]:
# Smooth L1 Loss
class SmoothL1Loss(nn.Module):

    def __init__(self, scale=0.01):
        super(SmoothL1Loss, self).__init__()
        self.scale = scale
        self.EPSILON = 1e-10

    def __repr__(self):
        return "SmoothL1Loss()"

    def forward(self, output: torch.Tensor, groundtruth: torch.Tensor, reduction='mean'):
        """
            input:  b x n x 2
            output: b x n x 1 => 1
        """
        if output.dim() == 4:
            shape = output.shape
            groundtruth = groundtruth.reshape(shape[0], shape[1], 1, shape[3])

        delta_2 = (output - groundtruth).pow(2).sum(dim=-1, keepdim=False)
        delta = delta_2.clamp(min=1e-6).sqrt()
        # delta = torch.sqrt(delta_2 + self.EPSILON)
        loss = torch.where(
            delta_2 < self.scale * self.scale,
            0.5 / self.scale * delta_2,
            delta - 0.5 * self.scale)

        if reduction == 'mean':
            loss = loss.mean()
        elif reduction == 'sum':
            loss = loss.sum()

        return loss

## 4.3. AWingLoss


In [17]:
# AWingLoss
class AWingLoss(nn.Module):
    
    def __init__(self, omega=14, theta=0.5, epsilon=1, alpha=2.1, use_weight_map=True):
        super(AWingLoss, self).__init__()
        self.omega = omega
        self.theta = theta
        self.epsilon = epsilon
        self.alpha = alpha
        self.use_weight_map = use_weight_map

    def __repr__(self):
        return "AWingLoss()"

    def generate_weight_map(self, heatmap, k_size=3, w=10):
        dilate = F.max_pool2d(heatmap, kernel_size=k_size, stride=1, padding=1)
        weight_map = torch.where(dilate < 0.2, torch.zeros_like(
            heatmap), torch.ones_like(heatmap))
        return w * weight_map + 1

    def forward(self, output, groundtruth):
        """
        input:  b x n x h x w
        output: b x n x h x w => 1
        """
        delta = (output - groundtruth).abs()
        A = self.omega * (1 / (1 + torch.pow(self.theta / self.epsilon, self.alpha - groundtruth))) * (self.alpha - groundtruth) * \
            (torch.pow(self.theta / self.epsilon, self.alpha -
             groundtruth - 1)) * (1 / self.epsilon)
        C = self.theta * A - self.omega * \
            torch.log(1 + torch.pow(self.theta /
                      self.epsilon, self.alpha - groundtruth))
        loss = torch.where(delta < self.theta,
                           self.omega *
                           torch.log(
                               1 + torch.pow(delta / self.epsilon, self.alpha - groundtruth)),
                           (A * delta - C))
        if self.use_weight_map:
            weight = self.generate_weight_map(groundtruth)
            loss = loss * weight
        return loss.mean()

## 4.4. STARLoss


In [18]:
# STAR Loss
def get_channel_sum(input):
    temp = torch.sum(input, dim=3)
    output = torch.sum(temp, dim=2)
    return output


def expand_two_dimensions_at_end(input, dim1, dim2):
    input = input.unsqueeze(-1).unsqueeze(-1)
    input = input.expand(-1, -1, dim1, dim2)
    return input


class STARLoss(nn.Module):
    
    def __init__(self, w=1, dist='smoothl1', num_dim_image=2, EPSILON=1e-5):
        super(STARLoss, self).__init__()
        self.w = w
        self.num_dim_image = num_dim_image
        self.EPSILON = EPSILON
        self.dist = dist
        if self.dist == 'smoothl1':
            self.dist_func = SmoothL1Loss()
        elif self.dist == 'l1':
            self.dist_func = F.l1_loss
        elif self.dist == 'l2':
            self.dist_func = F.mse_loss
        elif self.dist == 'wing':
            self.dist_func = WingLoss()
        else:
            raise NotImplementedError

    def __repr__(self):
        return "STARLoss()"

    def _make_grid(self, h, w):
        yy, xx = torch.meshgrid(
            torch.arange(h).float() / (h - 1) * 2 - 1,
            torch.arange(w).float() / (w - 1) * 2 - 1)
        return yy, xx

    def weighted_mean(self, heatmap):
        batch, npoints, h, w = heatmap.shape

        yy, xx = self._make_grid(h, w)
        yy = yy.view(1, 1, h, w).to(heatmap)
        xx = xx.view(1, 1, h, w).to(heatmap)

        yy_coord = (yy * heatmap).sum([2, 3])  # batch x npoints
        xx_coord = (xx * heatmap).sum([2, 3])  # batch x npoints
        coords = torch.stack([xx_coord, yy_coord], dim=-1)
        return coords

    def unbiased_weighted_covariance(self, htp, means, num_dim_image=2, EPSILON=1e-5):
        batch_size, num_points, height, width = htp.shape

        yv, xv = self._make_grid(height, width)
        xv = Variable(xv)
        yv = Variable(yv)

        if htp.is_cuda:
            xv = xv.cuda()
            yv = yv.cuda()

        xmean = means[:, :, 0]
        xv_minus_mean = xv.expand(batch_size, num_points, -1, -1) - expand_two_dimensions_at_end(xmean, height,
                                                                                                 width)  # [batch_size, 68, 64, 64]
        ymean = means[:, :, 1]
        yv_minus_mean = yv.expand(batch_size, num_points, -1, -1) - expand_two_dimensions_at_end(ymean, height,
                                                                                                 width)  # [batch_size, 68, 64, 64]
        wt_xv_minus_mean = xv_minus_mean
        wt_yv_minus_mean = yv_minus_mean

        wt_xv_minus_mean = wt_xv_minus_mean.view(
            batch_size * num_points, height * width)  # [batch_size*68, 4096]
        wt_xv_minus_mean = wt_xv_minus_mean.view(
            batch_size * num_points, 1, height * width)  # [batch_size*68, 1, 4096]
        wt_yv_minus_mean = wt_yv_minus_mean.view(
            batch_size * num_points, height * width)  # [batch_size*68, 4096]
        wt_yv_minus_mean = wt_yv_minus_mean.view(
            batch_size * num_points, 1, height * width)  # [batch_size*68, 1, 4096]
        # [batch_size*68, 2, 4096]
        vec_concat = torch.cat((wt_xv_minus_mean, wt_yv_minus_mean), 1)

        htp_vec = htp.view(batch_size * num_points, 1, height * width)
        htp_vec = htp_vec.expand(-1, 2, -1)

        # [batch_size*68, 2, 2]
        covariance = torch.bmm(htp_vec * vec_concat,
                               vec_concat.transpose(1, 2))
        covariance = covariance.view(
            batch_size, num_points, num_dim_image, num_dim_image)  # [batch_size, 68, 2, 2]

        V_1 = htp.sum([2, 3]) + EPSILON  # [batch_size, 68]
        V_2 = torch.pow(htp, 2).sum([2, 3]) + EPSILON  # [batch_size, 68]

        denominator = V_1 - (V_2 / V_1)
        covariance = covariance / \
            expand_two_dimensions_at_end(
                denominator, num_dim_image, num_dim_image)

        return covariance

    def ambiguity_guided_decompose(self, pts, eigenvalues, eigenvectors):
        batch_size, npoints = pts.shape[:2]
        rotate = torch.matmul(
            pts.view(batch_size, npoints, 1, 2), eigenvectors.transpose(-1, -2))
        scale = rotate.view(batch_size, npoints, 2) / \
            torch.sqrt(eigenvalues + self.EPSILON)
        return scale

    def eigenvalue_restriction(self, evalues, batch, npoints):
        eigen_loss = torch.abs(evalues.view(batch * npoints, 2)).sum(-1)
        return eigen_loss.mean()

    def forward(self, heatmap, groundtruth):
        """
            heatmap:     b x n x 64 x 64
            groundtruth: b x n x 2
            output:      b x n x 1 => 1
        """
        # normalize
        bs, npoints, h, w = heatmap.shape
        heatmap_sum = torch.clamp(heatmap.sum([2, 3]), min=1e-6)
        heatmap = heatmap / heatmap_sum.view(bs, npoints, 1, 1)

        means = self.weighted_mean(heatmap)  # [bs, 68, 2]
        covars = self.unbiased_weighted_covariance(
            heatmap, means)  # covars [bs, 68, 2, 2]

        # TODO: GPU-based eigen-decomposition
        # https://github.com/pytorch/pytorch/issues/60537
        _covars = covars.view(bs * npoints, 2, 2).cpu()
        # evalues [bs * 68, 2], evectors [bs * 68, 2, 2]
        evalues, evectors = _covars.symeig(eigenvectors=True)
        evalues = evalues.view(bs, npoints, 2).to(heatmap)
        evectors = evectors.view(bs, npoints, 2, 2).to(heatmap)

        # STAR Loss
        # Ambiguity-guided Decomposition
        error = self.ambiguity_guided_decompose(
            groundtruth - means, evalues, evectors)
        loss_trans = self.dist_func(torch.zeros_like(error).to(error), error)
        # Eigenvalue Restriction
        loss_eigen = self.eigenvalue_restriction(evalues, bs, npoints)
        star_loss = loss_trans + self.w * loss_eigen

        return star_loss

In [19]:
# [LOG]
print(">> [4] After defining loss functions!!!!")

>> [4] After defining loss functions!!!!


# 5. Metrics


## 5.1. Accuracy


In [20]:
# Accuracy
class Accuracy:
    
    def __init__(self):
        pass

    def __repr__(self):
        return "Accuracy()"

    def test(self, label_pd, label_gt, ignore_label=-1):
        correct_cnt = 0
        total_cnt = 0
        with torch.no_grad():
            label_pd = F.softmax(label_pd, dim=1)
            label_pd = torch.max(label_pd, 1)[1]
            label_gt = label_gt.long()
            c = (label_pd == label_gt)
            correct_cnt = torch.sum(c).item()
            total_cnt = c.size(0) - torch.sum(label_gt==ignore_label).item()
        return correct_cnt, total_cnt

## 5.2. FR_AUC


In [21]:
# FR_AUC
class FR_AUC:
    
    def __init__(self, data_definition):
        self.data_definition = data_definition
        if data_definition == '300W':
            self.thresh = 0.05
        else:
            self.thresh = 0.1

    def __repr__(self):
        return "FR_AUC()"

    def test(self, nmes, thres=None, step=0.0001):
        if thres is None:
            thres = self.thresh

        num_data = len(nmes)
        xs = np.arange(0, thres + step, step)
        ys = np.array([np.count_nonzero(nmes <= x) for x in xs]) / float(num_data)
        fr = 1.0 - ys[-1]
        auc = simps(ys, x=xs) / thres
        return [round(fr, 4), round(auc, 6)]

## 5.3. NME


In [22]:
# Normalized Mean Error
class NME:
    
    def __init__(self, nme_left_index, nme_right_index):
        self.nme_left_index = nme_left_index
        self.nme_right_index = nme_right_index

    def __repr__(self):
        return "NME()"

    def get_norm_distance(self, landmarks):
        assert isinstance(self.nme_right_index, list), 'the nme_right_index is not list.'
        assert isinstance(self.nme_left_index, list), 'the nme_left, index is not list.'
        right_pupil = landmarks[self.nme_right_index, :].mean(0)
        left_pupil = landmarks[self.nme_left_index, :].mean(0)
        norm_distance = np.linalg.norm(right_pupil - left_pupil)
        return norm_distance

    def test(self, label_pd, label_gt):
        nme_list = []
        label_pd = label_pd.data.cpu().numpy()
        label_gt = label_gt.data.cpu().numpy()

        for i in range(label_gt.shape[0]):
            landmarks_gt = label_gt[i]
            landmarks_pv = label_pd[i]
            if isinstance(self.nme_right_index, list):
                norm_distance = self.get_norm_distance(landmarks_gt)
            elif isinstance(self.nme_right_index, int):
                norm_distance = np.linalg.norm(landmarks_gt[self.nme_left_index] - landmarks_gt[self.nme_right_index])
            else:
                raise NotImplementedError
            landmarks_delta = landmarks_pv - landmarks_gt
            nme = (np.linalg.norm(landmarks_delta, axis=1) / norm_distance).mean()
            nme_list.append(nme)
            # sum_nme += nme
            # total_cnt += 1
        return nme_list

## 5.4. count_parameters_in_MB


In [23]:
# Count parameters in MB
def count_parameters_in_MB(model):
    if isinstance(model, nn.Module):
        return sum(v.numel() for v in model.parameters()) / 1e6
    else:
        return sum(v.numel() for v in model) / 1e6

In [24]:
# [LOG]
print(">> [5] After defining metrics!!!!")

>> [5] After defining metrics!!!!


# 6. Dataset


In [25]:
class Face300WXMLDataset(Dataset):
    """
    Dataset cho iBUG 300-W với XML annotations.
    """

    def __init__(self, root_dir, xml_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        xml_path = os.path.join(root_dir, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()
        for img in root.findall('images/image'):
            rel_path = img.get('file')
            img_path = os.path.join(root_dir, rel_path)
            parts = img.find('box').findall('part')
            lm = np.zeros((len(parts), 2), dtype=np.float16)
            for p in parts:
                idx = int(p.get('name'))
                lm[idx, 0] = np.float16(p.get('x'))
                lm[idx, 1] = np.float16(p.get('y'))
            self.samples.append({"img_path": img_path, "landmarks": lm})

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        image = Image.open(sample['img_path']).convert('RGB')
        landmarks = torch.from_numpy(sample['landmarks'])

        if self.transform:
            image = self.transform(image)

        return {"image": image, "landmarks": landmarks}

    # Apply the transform to all samples
    def apply_transform(self, transform) -> None:
        for sample in self.samples:
            image = Image.open(sample['img_path']).convert('RGB')
            sample['image'] = transform(image)
            sample['landmarks'] = torch.from_numpy(sample['landmarks']).float()

    # Set the transform for the dataset
    def set_transform(self, transform):
        self.transform = transform

## 6.1. Định nghĩa các transform


In [ ]:
# Training transforms với augmentations
train_transform = transforms.Compose([
    # Resize về 224×224
    # ⚙️ độ phân giải đầu vào :contentReference[oaicite:7]{index=7}
    transforms.Resize(size=(224,224), interpolation=InterpolationMode.BICUBIC),

    # Random affine: rotation ±18°, translate ±5%, scale [0.9,1.1]
    transforms.RandomAffine(
        # ±18° :contentReference[oaicite:8]{index=8}
        degrees=18,
        # ±5% kích thước :contentReference[oaicite:9]{index=9}
        translate=(0.05, 0.05),
        # ±10% scale :contentReference[oaicite:10]{index=10}
        scale=(0.9, 1.1)
    ),

    # Horizontal flip 50%
    # :contentReference[oaicite:11]{index=11}
    transforms.RandomHorizontalFlip(p=0.5),

    # Grayscale chuyển 20%
    # :contentReference[oaicite:12]{index=12}
    transforms.RandomGrayscale(p=0.2),

    # Gaussian blur 30%
    transforms.RandomApply(
        # or (3,3) :contentReference[oaicite:13]{index=13}
        [transforms.GaussianBlur(kernel_size=5)],
        p=0.3
    ),

    # Gamma adjustment 20%
    transforms.RandomApply(
        [transforms.Lambda(lambda img: transforms.functional.adjust_gamma(img,
                                                                          gamma=np.random.uniform(0.8, 1.2)))],
        p=0.2
    ),                                        # :contentReference[oaicite:14]{index=14}

    # Occlusion: RandomErasing 40%
    transforms.RandomErasing(p=0.4,
                             scale=(0.02, 0.25),
                             ratio=(0.3, 3.3),
                             value='random'),   # :contentReference[oaicite:15]{index=15}

    # Cuối cùng: ToTensor + Normalize ImageNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
val_test_transform = transforms.Compose([
    transforms.Resize(size=(224,224), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## 6.2. Khởi tạo DataLoaders


In [28]:
# Đường dẫn dataset
if os.path.exists('./ibug_300W_large_face_landmark_dataset'):
    root = './ibug_300W_large_face_landmark_dataset'
else:
    root = '/kaggle/input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset'
train_xml = 'labels_ibug_300W_train.xml'
test_xml = 'labels_ibug_300W_test.xml'

# Dataset
full_train_ds = Face300WXMLDataset(root, train_xml, transform=train_transform)
test_ds = Face300WXMLDataset(root, test_xml, transform=val_test_transform)

# Chia train/val 90/10, seed cố định
val_count = int(0.1 * len(full_train_ds))
train_count = len(full_train_ds) - val_count
generator = torch.Generator().manual_seed(0)
train_ds, val_ds = random_split(full_train_ds,
                                [train_count, val_count],
                                generator=generator)

# Set transform cho train/val/test
train_ds.dataset.set_transform(train_transform)
val_ds.dataset.set_transform(val_test_transform)
test_ds.set_transform(val_test_transform)

# DataLoader với batch_size=48
train_loader = DataLoader(train_ds, batch_size=48, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=48, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=48, shuffle=False)

In [29]:
# Iterate
for batch in train_loader:
    imgs = batch['image']       # (B,3,224,224)
    lms = batch['landmarks']    # (B,68,2)
    print(f"Train batch: images {imgs.shape}, landmarks {lms.shape}")
    break

# Print number of samples in each dataset
print(f"Train dataset size: {len(train_ds)}")
print(f"Validation dataset size: {len(val_ds)}")
print(f"Test dataset size: {len(test_ds)}")

Train batch: images torch.Size([48, 3, 224, 224]), landmarks torch.Size([48, 68, 2])
Train dataset size: 6000
Validation dataset size: 666
Test dataset size: 1008


In [30]:
# [LOG]
print(">> [6] After defining dataset and dataloader!!!!")

>> [6] After defining dataset and dataloader!!!!


# 7. Training Model


## 7.1. Configuration


In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 12
batch_size = 48
initial_lr = 1e-4
weight_decay = 1e-5
milestones = [6, 10]

In [32]:
# Print the used device
print(f"Using device: {device}")

Using device: cuda


## 7.2. Model, Losses, Metrics


In [33]:
# Backbone choice: 'mobilenet', 'resnet101', 'convnext_b', 'swin_b'
def get_model(backbone_name='swin_b'):
    model = FaceXFormer(backbone_name=backbone_name)

    # Initialize the parameters
    for p in model.parameters():
        if p.dim() > 1:
            # Add seed for reproducibility
            nn.init.xavier_uniform_(
                p, generator=torch.Generator().manual_seed(0))
        else:
            nn.init.zeros_(p)

    return model.to(device)


# Load the model
backbone_name: str = 'swin_b'
model = get_model(backbone_name=backbone_name)


# Landmark loss
landmark_loss_fn = WingLoss().to(device)

# Metrics
nme_metric = NME(nme_left_index=[36, 39],
                 nme_right_index=[42, 45])  # eye landmarks for normalization
# uses simps for AUC :contentReference[oaicite:3]{index=3}
fr_auc_metric = FR_AUC(data_definition='300W')

In [34]:
# Enable CUDA optimization for half precision (if available)
if device.type == 'cuda':
    # Enable autocast for mixed precision training
    scaler = torch.amp.GradScaler()
    print("CUDA enabled with automatic mixed precision")
else:
    scaler = None
    print("Running on CPU, mixed precision not enabled")

# Report memory usage
if device.type == 'cuda':
    print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"Model size: {count_parameters_in_MB(model):.2f} MB")

CUDA enabled with automatic mixed precision
GPU memory allocated: 0.37 GB
Model size: 90.96 MB


## 7.3. Optimizer and Scheduler


In [35]:
optimizer = AdamW(
    model.parameters(),
    lr=initial_lr,
    weight_decay=weight_decay
)
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.1)

## 7.4. Training & Validation Loop


In [36]:
def denorm_points(points, h, w, align_corners=False):
    if align_corners:
        denorm_points = (points + 1) / 2 * torch.tensor([w - 1, h - 1], dtype=torch.float32).to(points).view(1, 1, 2)
    else:
        denorm_points = ((points + 1) * torch.tensor([w, h], dtype=torch.float32).to(points).view(1, 1, 2) - 1) / 2

    return denorm_points

In [ ]:
best_nme = float('inf')
best_model_wts = None

for epoch in range(1, num_epochs + 1):
    epoch_start = time.time()

    # --- Training Phase ---
    model.train()
    running_loss = 0.0
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} [Train]")
    for batch in train_pbar:
        imgs = batch['image'].to(device)
        targets = batch['landmarks'].to(device, dtype=torch.float32)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device.type, dtype=torch.float16):
            preds = model(imgs, labels=None, tasks=None)
            denorm_lms = denorm_points(preds.view(-1, 68, 2), 224, 224)
            loss = landmark_loss_fn(denorm_lms, targets)

        # Scales loss. Calls backward() on scaled loss to create scaled gradients.
        scaler.scale(loss).backward()
        # Unscales gradients and calls optimizer.step() if gradients are not NaN
        scaler.step(optimizer)
        # Updates the scale for next iteration
        scaler.update()

        running_loss += loss.item() * imgs.size(0)
        train_pbar.set_postfix(
            {'loss': f"{loss.item():.4f}", 'lr': f"{optimizer.param_groups[0]['lr']:.6f}"})

    train_time = time.time() - epoch_start
    train_loss = running_loss / len(train_loader.dataset)

    # Free memory
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # --- Validation Phase ---
    model.eval()
    nmes = []
    val_pbar = tqdm(val_loader, desc=f"Epoch {epoch}/{num_epochs} [Valid]")
    with torch.no_grad():
        for batch in val_pbar:
            imgs = batch['image'].to(device)
            targets = batch['landmarks'].to(device, dtype=torch.float32)

            with torch.amp.autocast(device_type=device.type, dtype=torch.float16):
                preds = model(imgs, labels=None, tasks=None)
                denorm_lms = denorm_points(preds.view(-1, 68, 2), 224, 224)
                batch_nmes = nme_metric.test(denorm_lms, targets)

            nmes.extend(batch_nmes)
            current_nme = sum(batch_nmes) / \
                len(batch_nmes) if batch_nmes else 0
            val_pbar.set_postfix({'NME': f"{current_nme:.4f}"})

    mean_nme = sum(nmes) / len(nmes)

    # Free memory
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Scheduler step - this is correct as we're doing it once per epoch
    # after all optimizer steps are completed
    scheduler.step()

    # Check for best
    if mean_nme < best_nme:
        # Free memory
        del best_model_wts
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # Save the best model
        best_nme = mean_nme
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"✅ New best model saved! NME: {mean_nme:.4f}")

    print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} "
          f"| Val NME: {mean_nme:.4f} | Train Time: {train_time:.1f}s")

Epoch 1/12 [Valid]: 100%|██████████| 14/14 [00:22<00:00,  1.63s/it, NME=4.7109]


✅ New best model saved! NME: 4.2634
Epoch 01 | Train Loss: 726.7489 | Val NME: 4.2634 | Train Time: 277.5s


Epoch 2/12 [Valid]: 100%|██████████| 14/14 [00:17<00:00,  1.26s/it, NME=4.6963]


✅ New best model saved! NME: 4.2479
Epoch 02 | Train Loss: 724.9433 | Val NME: 4.2479 | Train Time: 233.1s


Epoch 3/12 [Valid]: 100%|██████████| 14/14 [00:17<00:00,  1.26s/it, NME=4.6820]


...

✅ New best model saved! NME: 4.1790
Epoch 10 | Train Loss: 716.2188 | Val NME: 4.1790 | Train Time: 230.6s


Epoch 11/12 [Valid]: 100%|██████████| 14/14 [00:17<00:00,  1.25s/it, NME=4.6313]


✅ New best model saved! NME: 4.1788
Epoch 11 | Train Loss: 716.0880 | Val NME: 4.1788 | Train Time: 228.5s


Epoch 12/12 [Valid]: 100%|██████████| 14/14 [00:17<00:00,  1.25s/it, NME=4.6312]

✅ New best model saved! NME: 4.1788
Epoch 12 | Train Loss: 716.0749 | Val NME: 4.1788 | Train Time: 229.1s


In [38]:
# [LOG]
print(">> [7] After training the model!!!!")

>> [7] After training the model!!!!


# 8. Final Evaluation on Test Set


In [39]:
# Load best model
model.load_state_dict(best_model_wts)
model.eval()

# Evaluate on test
test_nmes = []
inference_times = []
with torch.no_grad():
    test_pbar = tqdm(test_loader, desc="[Testing]")
    for batch in test_pbar:
        imgs = batch['image'].to(device)
        targets = batch['landmarks'].to(device, dtype=torch.float32)

        with torch.amp.autocast(device_type=device.type, dtype=torch.float16):
            start = time.time()
            preds = model(imgs, labels=None, tasks=None)
            end = time.time()

            denorm_lms = denorm_points(preds.view(-1, 68, 2), 224, 224)
            test_nmes.extend(nme_metric.test(denorm_lms, targets))

        inference_times.append((end - start) / imgs.size(0))

# Metrics
mean_test_nme = sum(test_nmes) / len(test_nmes)
# [FR, AUC] using Simpson's rule :contentReference[oaicite:9]{index=9}
fr, auc = fr_auc_metric.test(test_nmes)
mean_inference_time = sum(inference_times) / len(inference_times)

print("\nTest Results:")
print(f"  Mean NME    : {mean_test_nme:.4f}")
print(f"  FR (@0.05)  : {fr:.4f}")
print(f"  AUC         : {auc:.6f}")
print(f"  Inference/s : {1/mean_inference_time:.1f} img/s")

[Testing]: 100%|██████████| 21/21 [00:35<00:00,  1.67s/it]


Test Results:
  Mean NME    : 4.2830
  FR (@0.05)  : 1.0000
  AUC         : 0.000000
  Inference/s : 479.7 img/s


In [40]:
# [LOG]
print(">> [8] After evaluating the model on test set!!!!")

>> [8] After evaluating the model on test set!!!!


# 9. Save best model


In [41]:
# Create directory for saving models if it doesn't exist
save_dir = './models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)

# Save the best model weights
model_save_path = os.path.join(
    save_dir, f'facexformer_{backbone_name}_best.pth')
torch.save({
    # Save backbone name
    'backbone_name': backbone_name,
    # Save model state dict
    'state_dict': best_model_wts,
    # Save metric values on test set
    'metrics': {
        'nme': mean_test_nme,
        'fr': fr,
        'auc': auc,
        'inference_time': mean_inference_time
    },
}, model_save_path)

print(f"Best model saved to {model_save_path}")
print(
    f"Model metrics - NME: {mean_test_nme:.4f}, FR@0.05: {fr:.4f}, AUC: {auc:.6f}")

Best model saved to ./models/facexformer_swin_b_best.pth
Model metrics - NME: 4.2830, FR@0.05: 1.0000, AUC: 0.000000


# 10. Load best model


In [42]:
# Load the best model with exact backbone
def load_facexformer_model(model_path):
    # Check if file exists
    if not os.path.exists(model_path):
        print(f"Model file not found at {model_path}")
        return None, None

    # Load model checkpoint
    checkpoint = torch.load(model_path, map_location=device,
                            weights_only=False)
    backbone_name = checkpoint['backbone_name']

    # Initialize model with the same backbone
    model = FaceXFormer(backbone_name=backbone_name)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    model.eval()

    # Get metrics
    metrics = checkpoint['metrics']

    print(f"Loaded FaceXFormer model with {backbone_name} backbone")
    print(f"Metrics on test set:")
    print(f"  Mean NME    : {metrics['nme']:.4f}")
    print(f"  FR (@0.05)  : {metrics['fr']:.4f}")
    print(f"  AUC         : {metrics['auc']:.6f}")
    print(f"  Inference/s : {1/metrics['inference_time']:.1f} img/s")

    return model, metrics


# Load the best model
model_path = os.path.join(save_dir, f'facexformer_{backbone_name}_best.pth')
loaded_model, loaded_metrics = load_facexformer_model(model_path)

Loaded FaceXFormer model with swin_b backbone
Metrics on test set:
  Mean NME    : 4.2830
  FR (@0.05)  : 1.0000
  AUC         : 0.000000
  Inference/s : 479.7 img/s
